In [1]:
from transformers.convert_graph_to_onnx import convert
from transformers import AutoTokenizer

In [2]:
tokenizer = AutoTokenizer.from_pretrained("hfl/chinese-electra-180g-large-discriminator", use_fast=True)

In [3]:
from pathlib import Path

# Handles all the above steps for you
convert(
        framework="pt", 
        model='ckpt/step-02/checkpoint-1149', 
        output=Path("../../deploy/zh-sentiment-tagging-2/model.onnx"), 
        opset=11,
        tokenizer=tokenizer,
        pipeline_name='ner'
       )

ONNX opset version set to: 11
Loading pipeline (model: ckpt/step-02/checkpoint-1149, tokenizer: PreTrainedTokenizerFast(name_or_path='hfl/chinese-electra-180g-large-discriminator', vocab_size=21128, model_max_len=1000000000000000019884624838656, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}))
Creating folder ../../deploy/zh-sentiment-tagging-2
Using framework PyTorch: 1.6.0+cu101
Found input input_ids with shape: {0: 'batch', 1: 'sequence'}
Found input token_type_ids with shape: {0: 'batch', 1: 'sequence'}
Found input attention_mask with shape: {0: 'batch', 1: 'sequence'}
Found output output_0 with shape: {0: 'batch', 1: 'sequence'}
Ensuring inputs are in correct order
position_ids is not present in the generated input list.
Generated inputs order: ['input_ids', 'attention_mask', 'token_type_ids']


/usr/local/lib/python3.7/dist-packages/transformers/modeling_electra.py:180: TracerWarning: Converting a tensor to a Python index might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  position_ids = self.position_ids[:, :seq_length]
/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1673: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  input_tensor.shape == tensor_shape for input_tensor in input_tensors


---

In [1]:
from os import environ
from psutil import cpu_count

# Constants from the performance optimization available in onnxruntime
# It needs to be done before importing onnxruntime
environ["OMP_NUM_THREADS"] = str(cpu_count(logical=True))
environ["OMP_WAIT_POLICY"] = 'ACTIVE'

from onnxruntime import GraphOptimizationLevel, InferenceSession, SessionOptions, get_all_providers

In [2]:
from contextlib import contextmanager
from dataclasses import dataclass
from time import time
from tqdm import trange

def create_model_for_provider(model_path: str, provider: str) -> InferenceSession: 
  
    assert provider in get_all_providers(), f"provider {provider} not found, {get_all_providers()}"

    # Few properties that might have an impact on performances (provided by MS)
    options = SessionOptions()
    options.intra_op_num_threads = 1
    options.graph_optimization_level = GraphOptimizationLevel.ORT_ENABLE_ALL

    # Load the model as a graph and prepare the CPU backend 
    session = InferenceSession(model_path, options, providers=[provider])
    session.disable_fallback()

    return session


@contextmanager
def track_infer_time(buffer: [int]):
    start = time()
    yield
    end = time()

    buffer.append(end - start)


@dataclass
class OnnxInferenceResult:
    model_inference_time: [int]  
    optimized_model_path: str

In [3]:
from transformers import ElectraTokenizerFast

In [4]:
tokenizer = ElectraTokenizerFast.from_pretrained("hfl/chinese-electra-180g-large-discriminator")

In [6]:
cpu_model = create_model_for_provider("onnx/zh-sentiment-tagging-test", "CPUExecutionProvider")

In [7]:
model_inputs = tokenizer("是从深圳发出的，感觉香味有点涩，持续时间还可以。", return_tensors="pt")

In [8]:
inputs_onnx = {k: v.cpu().detach().numpy() for k, v in model_inputs.items()}

# Run the model (None = get all the outputs)
result = cpu_model.run(None, inputs_onnx)

In [11]:
impor numpy

SyntaxError: invalid syntax (<ipython-input-11-600c8a62b1e5>, line 1)

In [14]:
len(result)

1

In [19]:
result[0].argmax(-1)

array([[2, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 4, 4, 4, 4, 4, 0, 1, 2, 2, 2, 2,
        2, 2, 2, 0]])